<a href="https://colab.research.google.com/github/zi-onion/ESAA/blob/main/04_01_Evaluation(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 06 피마 인디언 당뇨병 예측(Pima IndianDiabetes)
: 북아메리카 피마지역 원주민의 Type-2 당뇨병 결과 데이터


* Pregnancies: 임신 횟수
* Glucose: 포도당 부하 검사 수치
* BloodPressures: 혈압(mm Hg)
* SkinThickness: 팔 상두근 뒤쪽의 피하지방 측정값(mm)
* Insulin: 혈청 인슐린(mu U/ml)
* BMI: 체질량지수(체중(kg)/키(m))^2)
* DiabetesPedigreeFunction: 당뇨 내력 가중치 값
* Age: 나이
* Outcome: 클래스 결정값 (0또는 1)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

diabetes_data = pd.read_csv('diabetes.csv')
print(diabetes_data['Outcome'].value_counts())
diabetes_data.head(3)

In [ ]:
#feature의 타입과 Null 개수 확인
diabetes_data.info()

In [ ]:
# get_clf_eval 함수 정의
def get_clf_eval(y_test, pred=None, pred_proba=None) :
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    # ROC-AUC 추가
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # roc_auc 추가
    print('정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}, \
    F1 : {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))


In [ ]:
# 로지스틱 회귀를 이용해 예측 모델을 생성

# 피처 데이터 세트 X, 레이블 데이터 세트 y를 추출
# 맨 끝이 Outcome 칼럼으로 레이블 값임. 칼럼 위치 -1을 이용해 추출
X = diabetes_data.iloc[:,:-1]
y = diabetes_data.iloc[:, -1]

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size =0.2, random_state=156, stratify=y)

# 로지스틱 회귀로 학습, 예측 및 평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
pred_proba = lr_clf.predict_proba(X_test)[:,1]

get_clf_eval(y_test, pred, pred_proba)

-> 예측정확도가 77.27%, 재현율은 59. 26%로 측정. 전체 데이터의 65%가 Negative이므로 정 확도보다는 재현율 성능에 조금 더 초점을 맞추기.


In [ ]:
#precision_recall_curve_plot 함수 정의

def precision_recall_curve_plot(y_test, pred_proba_c1):
    # threshold ndarray와 이 threshold에 정밀도, 재현율 ndarray 추출.
    precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_c1)

    # X축을 threshold값으로, Y축은 정밀도, 재현율 값으로 각각 PLot 수행. 정밀도는 점선으로 표시
    plt.figure(figsize = (8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label = 'precision')
    plt.plot(thresholds, recalls[0:threshold_boundary], label='recall')

    # threshold 값 X축의 Scale을 0.1 단위로 변경
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))

    # x축, y축 label과 legend, 그리고 grid 설정
    plt.xlabel('Threshold value'); plt.ylabel('Precision and Recall value')
    plt.legend(); plt.grid()
    plt.show()

In [ ]:
# 정밀도 재현율 곡선을 보고 임곗값 별 정밀도와 재현율 값의 변화 확인
pred_proba_c1 = lr_clf.predict_proba(X_test)[:,1]
precision_recall_curve_plot(y_test, pred_proba_c1)

임곗값을 0.42 정도로 낮추면 정밀도와 재현율의 균형이 맞음. 그러나 그마저도 0.7이 안되는 수치로 낮음

--> 임곗값을 인위적으로 조작하기 전에 다시 데이터 값을 점검

In [ ]:
diabetes_data.describe()

min()값이 0인 피처가 너무 많음 -> 말이 안됨.
ex)Glucose

In [ ]:
plt.hist(diabetes_data['Glucose'], bins = 10)

In [ ]:
# 확인할 피처: Glucose, BloodPressure,SkinThickness, Tnsulin, BMI

# 값을 검사할 피처명 리스트
zero_features = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# 전체 데이터 건수
total_count = diabetes_data['Glucose'].count()

# 피처별로 반복하면서 데이터 값이 0인 데이터 건수를 추출하고, 퍼센트 계산
for feature in zero_features:
    zero_count = diabetes_data[diabetes_data[feature]==0][feature].count()
    print('{0} 0 건수는 {1}, 퍼센트는 {2:.2f} %'.format(feature, zero_count,
                                                 100*zero_count/total_count))

데이터 건수가 적기에 피처의 0값을 평균값으로 대체

In [ ]:
# zero_features 리스트 내부에 저장된 개별 피처들에 대해서 0값을 평균값으로 대체
mean_zero_features = diabetes_data[zero_features].mean()
diabetes_data[zero_features] = diabetes_data[zero_features].replace(0, mean_zero_features)

In [ ]:
# 피처 스케일링을 적용해 변환

X = diabetes_data.iloc[:,:-1]
y = diabetes_data.iloc[:,-1]

# StandardScaler 클래스를 이용해 피처 데이터 세트에 일괄적으로 스케일링 적용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state=156, stratify=y)

# 로지스틱 회귀로 학습, 예측 및 평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
pred_proba = lr_clf.predict_proba(X_test)[:,1]

get_clf_eval(y_test, pred, pred_proba)

여전히 재현율 수치는 개선이 필요함

In [ ]:
# get_eval_by_threshold 함수 정의
from sklearn.preprocessing import Binarizer

def get_clf_eval(y_test, pred) :
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차 행렬')
    print(confusion)

    print('정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}, f1 : {3:.4f}'.format(accuracy, precision, recall, f1))
def get_eval_by_threshold(y_test, pred_proba_c1, thresholds) :
    for custom_threshold in thresholds :
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('임곗값', custom_threshold)
        get_clf_eval(y_test, custom_predict)


# 임곗값을 0.3에서 0.5까지 0.03씩 변화시키면서 재현율과 다른 평가 지표의 값 변화를 출력

thresholds = [0.3, 0.33, 0.36, 0.39, 0.42, 0.45, 0.48, 0.50]
pred_proba = lr_clf.predict_proba(X_test)
get_eval_by_threshold(y_test, pred_proba[:, 1].reshape(-1,1), thresholds)


정확도와 정밀도를 희생하고 재현율을 높이는데 가장 좋은 임곗값은 0.33으로, 재현율 값이 0.7963임.그러나 정밀도는 0.5972 -> 저조

임곗값 0.48이 전체적인 성능 평가 지표를 유지하면서 재현율을 약간 향상시키는 좋은 임곗값이라 판단.

이 때 정확도는 0.7987, 정밀도는 0.747, 재현율은 0.6481, F1스코어는 0.6931, ROC AUC는 0.843





In [ ]:
# 로지스틱 회귀 모델을 이용해 임곗값을 0.48로 낮춘 상태에서 다시 예측
# Binarizer 클래스를 이용해predict_proba( )사용

# 임곗값을 0.48로 설정한 Binarizer 생성
binarizer = Binarizer(threshold=0.48)

# 위에서 구한 lr_clf와 predict_proba() 예측 확률 array에서 1에 해당하는 칼럼값을 Binarizer 변환.
pred_th_048 = binarizer.fit_transform(pred_proba[:,1].reshape(-1,1))

get_clf_eval(y_test, pred_th_048, pred_proba[:,1])

## 07 정리

이진 분류의 레이블 값이 불균형하게 분포될 경우 단순히 정확도만으로는 머신러닝 모델의 예측 성능을 평가할 수 없음.

* 오차행렬
: Negative와 Positive값을 가지는 실제 클래스 값과 예측 클래스 값이 True와 False에 따라 TN, FP, FN,TP로 매핑되는 4분면 행렬을 기반으로 예측 성능을 평가
* 정밀도(Precision)와 재현율(Recall)은 Positive 데이터 세트의 예측 성능에 좀 더 초점을 맞춘 평가지표.  
 * 재현율은 암양성 예측모델과 같이 실제 Positive 양성인 데이터 예측을 Negative로 잘못 판단하게 되면 업무상 큰 영향이 발생하는 경우 상대적으로 더 중요한 지표로 쓰임.
 * 정밀도 또는 재현율이 특별히 강조돼야 할 경우 분류의 결정 임곗값(Threshold)을 조정해 수치를 높일 수 있음.

-> 이를 통해 분류모델 예측 성능의 오류가 어떠한 모습으로 발생하는지 알 수 있음.


* F1 스코어: 정밀도와 재현율을 결합한 평가지표.
정밀도와 재현율이 균형을 이룰 때 높은 지표값 가짐. 을가지게됩니다.
* ROC-AUC: 일반적으로 이진 분류의 성능 평가를 위해 가장 많이 사용되는 지표
 * AUC(Area Under Curve): ROC 곡선 밑의 면적을 구한 것으로서 일반 적으로 1에 가까울수록 좋음.